In [55]:
import pandas as pd
import requests

import requests
from bs4 import BeautifulSoup

from bs4.element import Comment

import pandas as pd
import re
from tqdm import trange, tqdm
from pathlib import Path

In [6]:

cd /Users/kamil/dev/nlp/TatarTopicModeling

/Users/kamil/dev/nlp/TatarTopicModeling


In [33]:
cats_path  = 'data/raw/cats.csv'
df = pd.read_csv(cats_path)

# df = df[df['count']>0]
df.sort_values('count', ascending=False, inplace=True)

df['title'] = df['title'].str.lower()
stop = ['торак пункт', 'елга', 'фамилия', 'википедия', 'wikidata', 'туу', 'үлем', 'казан', 'уфа', 'вафат', 'сурәтләр', 'саннар', 'авыл']
for s in stop:
    df = df[ ~df['title'].str.contains(s) ]

df

,title,url,count
22337,хронологик исемлекләр,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,3926
13957,еллар,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,3126
13971,еллар буенча эшләп бетерелмәгән мәкаләләр,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,2764
16500,күп мәгънәле мәкаләләр,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,2694
12301,билгесез параметрларны куллана торган битләр (...,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,2429
...,...,...,...
7370,471 елда туганнар,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,0
7374,472 елда туганнар,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,0
7378,473 елда туганнар,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,0
7382,474 елда туганнар,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,0


In [34]:
topics = ['Астрономия', 'Биология', 'Химия', 'Физика', 'Математика', 'География', 'тарих', 
          'әдәбият', 'Фәлсәфә', 'Психология', 'Сәясәт', 'Икътисад', 'хокук']
topics = [t.lower() for t in topics]
total_art = 0

df_cat = pd.DataFrame(columns=['topic','title','url','count'])

for t in topics:
    cat = df[df['title'].str.contains(t)]
    cat['topic'] = t
    cnt = cat["count"].sum()
    total_art += cnt
    df_cat = df_cat.append(cat)
    print(f'{t}, \t cats: {len(cat)}, \t articles: {cnt}')

print()
print('Total articles:', total_art)
df_cat

астрономия, 	 cats: 9, 	 articles: 84
биология, 	 cats: 25, 	 articles: 133
химия, 	 cats: 22, 	 articles: 219
физика, 	 cats: 31, 	 articles: 475
математика, 	 cats: 12, 	 articles: 116
география, 	 cats: 237, 	 articles: 1088
тарих, 	 cats: 258, 	 articles: 2297
әдәбият, 	 cats: 82, 	 articles: 568
фәлсәфә, 	 cats: 64, 	 articles: 406
психология, 	 cats: 12, 	 articles: 46
сәясәт, 	 cats: 121, 	 articles: 889
икътисад, 	 cats: 49, 	 articles: 408
хокук, 	 cats: 35, 	 articles: 194

Total articles: 6923


/Users/kamil/dev/nlp/TatarTopicModeling/venv/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,topic,title,url,count
11644,астрономия,астрономия,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,49
15320,астрономия,калыплар:астрономия,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,17
24144,астрономия,һинд дине астрономиясе,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,7
17666,астрономия,навигацион калыплар:астрономия,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,5
13402,астрономия,гастрономия,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,3
...,...,...,...,...
22035,хокук,франция хокук белгечләре,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,1
22292,хокук,хокук бозучылар,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,1
14572,хокук,интеллектуаль милек хокукы,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,1
14681,хокук,ислам хокукы чыганаклары,https://tt.wikipedia.org/wiki/%D0%A2%D3%A9%D1%...,1


In [56]:
from functools import reduce

BASE_URL = 'https://tt.wikipedia.org/'

def filter_url(url: str):
    _stop = ['%D0%A2%D3%A9%D1%80%D0%BA%D0%B5%D0%BC']
    cond = [s not in url for s in _stop]
    cond = reduce(lambda x, y: x * y, cond)
    return bool(cond)


def process_cat(page_url):
    cont = requests.get(page_url).content
    soup = BeautifulSoup(cont)

    articles_block = soup.find('div', dir='ltr', class_='mw-content-ltr')
    links = articles_block.find_all('a', class_=None)
    links = [(l.get('title'), l.get('href')) for l in links]

    links = list(set(links))
    links = [l for l in links if filter_url(l[1])]

    df = pd.DataFrame(columns=['title', 'url'])
    df['title'] = [l[0].strip() for l in links]
    df['url'] = [BASE_URL + l[1] for l in links]

    return df

i = 0
res_df = pd.DataFrame(columns = ['topic', 'cat','title', 'url'])
for idx, row in tqdm(df_cat.iterrows()):
    df = process_cat(row['url'])
    df['topic'] = row['topic']
    df['cat'] = row['title']
    res_df = res_df.append(df)
#     i+=1
#     if i>5:
#         break
    
res_df.to_csv(f'data/raw/titles.csv', index=False)
res_df

957it [07:57,  2.00it/s]


,topic,cat,title,url
0,астрономия,астрономия,Гаусс елы,https://tt.wikipedia.org//wiki/%D0%93%D0%B0%D1...
1,астрономия,астрономия,Кара тишек,https://tt.wikipedia.org//wiki/%D0%9A%D0%B0%D1...
2,астрономия,астрономия,Галактика,https://tt.wikipedia.org//wiki/%D0%93%D0%B0%D0...
3,астрономия,астрономия,Habbl qanunı,https://tt.wikipedia.org//wiki/Habbl_qanun%C4%B1
4,астрономия,астрономия,Галактик җеп,https://tt.wikipedia.org//wiki/%D0%93%D0%B0%D0...
...,...,...,...,...
4,хокук,татар хокукчылары,Портал:Татар мәдәнияте,https://tt.wikipedia.org//wiki/%D0%9F%D0%BE%D1...
5,хокук,татар хокукчылары,Портал:Шәхесләр,https://tt.wikipedia.org//wiki/%D0%9F%D0%BE%D1...
0,хокук,рф президенты каршында балалар хокуклары буенч...,Анна Кузнецова,https://tt.wikipedia.org//wiki/%D0%90%D0%BD%D0...
0,хокук,франция хокук белгечләре,Шарль де Монтескьё,https://tt.wikipedia.org//wiki/%D0%A8%D0%B0%D1...


In [75]:
import wikipedia
wikipedia.set_lang('tt')

df = pd.read_csv(f'data/raw/titles.csv')

cont_path = f'data/raw/contents.csv'

odf = pd.DataFrame(columns = ['topic', 'cat','title', 'url', 'content'])
odf.to_csv(cont_path, mode='a', index=False, sep = '\n')

for idx, row in tqdm(df.iterrows()):   
    try:
        title = row['title'].lower()
#       title = '_'.join(title.split())
        page = wikipedia.page(title =title)
        row['content']=page.content.strip()
    
        row.to_csv(cont_path, mode='a', index=False, sep = '\n')
    
    except Exception as e:
        pass


1912it [59:56,  1.52s/it]/Users/kamil/dev/nlp/TatarTopicModeling/venv/lib/python3.7/site-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/kamil/dev/nlp/TatarTopicModeling/venv/lib/python3.7/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
4888it [2:21:05,  1.73s/it]


In [ ]:
r = wikipedia.search('гаусс елы')
r